## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

 Описание данных

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter
import math

In [2]:
data=pd.read_csv('/datasets/data.csv')

Рассмотрим данные в начале и конце таблицы, и общую информацию с помощью .head(),.tail(),.info(),.describe()

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.tail(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21506,1,-1556.249906,33,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145541.993320,свадьба
21507,1,-79.832064,32,среднее,1,гражданский брак,1,F,госслужащий,0,98180.279152,сделка с подержанным автомобилем
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод


1. Нужно заменить названия некоторых столбцов на более удобные в работе

2. В столбцах 'days_employed' и 'total_income' есть пропуски данных.Их нужно заполнить

3. В столбце 'children' диапазон значений от -1 до 20. Возможно всего эти данные ошибочны, и необходимо будет  проверить, сколько всего таких строк и затем принять решение об их коррекции или удалении.

4. В столбце 'days_employed' есть отрицательные значения. Возможно эти данные можно будет использовать в исследовании, взяв их по модулю.

4. Общий трудовой стаж удобнее учитывать в годах, а не днях, можно заменить формат вывода данных в столбце

5. Видно, что данные в некоторых столбцах дублируют друг-друга('education' -   'education_id',и 'family_status'- 'family_status_id' Нужно привести все буквы в нижний регистр в столбцах 'family_status' и 'education'




### Перед анализом данных нужно провести их предварительную обработку, очистить от пропусков, заменить тип данных,избавить от дубликатов,провести лемматизацию и категоризацию данных  
    

### Шаг 2. Предобработка данных

### Обработка пропусков

Для удобства работы переименуем некоторые столбцы

In [7]:
#для переименования столбцов воспользуемся методом set_axis()
columns = ['number_of_children',
        'employment_term',
        'age',
        'education_level',
        'education_level_id',
        'family_status',
        'family_status_id',
        'gender',
        'income_type',
        'arrear_status',
        'monthly_income',
        'credit_purpose'
       ]
data.set_axis(columns, axis='columns', inplace=True)

Методом isnull() найдём все строки с пропусками в столбце 'monthly_income' и просмотрим первые 20 строк таблицы. 

In [8]:
data[data['monthly_income'].isnull()].head(20)

,number_of_children,employment_term,age,education_level,education_level_id,family_status,family_status_id,gender,income_type,arrear_status,monthly_income,credit_purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Создается ощущение, что данные в столбцах 'employment_term' и 'monthly_income' отсутствуют в одинаковых строках. Проверим это предположение. Посчитаем суммарное количество прропусков.

In [9]:
data.isnull().sum()

number_of_children       0
employment_term       2174
age                      0
education_level          0
education_level_id       0
family_status            0
family_status_id         0
gender                   0
income_type              0
arrear_status            0
monthly_income        2174
credit_purpose           0
dtype: int64

Данные в столбцах 'employment_term' и 'monthly_income' отсутствуют в одних и тех же строках. Возможно, что эти люди подавали заявку на кредит по упрощенной схеме, без подтверждения стажа и дохода, например кредит по 2 документам.Или мы имеем дело просто со сбоем системы, в ходе которого часть данных потерялась.

Посмотрим на количество строк с нелогичными данными (артефактами) в столбцах 'employment_term','number_of_children', 'age'.

In [10]:
print("Количество строк, где возраст равен 0:", data[data['age'] == 0].count()[0])
print("Количество строк, где количество детей меньше 0:", data[data['number_of_children'] == -1].count()[0])
print("Количество строк, где количество детей равно 20:", data[data['number_of_children'] == 20].count()[0])

Количество строк, где возраст равен 0: 101
Количество строк, где количество детей меньше 0: 47
Количество строк, где количество детей равно 20: 76


Нужно будет внести исправления в столбец 'age', так как возраст клиентов для выдачи кредита явно не может равняться 0, но скорее всего эти данные отсутствуют в базе, нужно будет запросить дополнительную информацию

Строки, где количество детей меньше нуля, нужно взять по модулю.


In [11]:
data['employment_term']=abs(data['employment_term'])
data['number_of_children']=abs(data['number_of_children'])

Строки, где количество детей больше 20 - это не исключительный случай - их 76 в датафрейме, скорее всего это опечатка при заполнении, и 0 можно отбросить, таким образом количество детей  этих строках будет равно 2  

In [12]:
data['number_of_children'] = data['number_of_children'].replace(20, 2)

Проверим результат, убедимся, что артефакты исчезли

In [13]:
data.describe()

,number_of_children,employment_term,age,education_level_id,family_status_id,arrear_status,monthly_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.479721,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,0.755528,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Рассчитаем среднюю (mean) для столбца 'employment_term' и медиану для столбца 'monthly_income' и заменим отсутсвующие значения в столбце 'employment_term' на среднюю для всего столбца, отсутсвующие значения в столбце 'monthly_income' на медиану этого столбца. Выведем результат, чтобы убедиться, что строк с пропущенными значениями в таблице больше не осталось

In [14]:
employment_term_mean = data['employment_term'].mean()
data['employment_term'] = data['employment_term'].fillna(employment_term_mean)

## <font color=purple>Рассчитаем медианы по доходу в зависимости от рода деятельности и заполним пропуски получившимися данными.

In [15]:
#Рассчитываем медианы по типам дохода
monthly_income_median = data.groupby('income_type')['monthly_income'].aggregate('median')
# Определяем функцию, устанавливающую значение 'monthly_income' по соответствующей медиане, 
# если значение 'monthly_income' не определено
def pushNa(row):
    if math.isnan(row['monthly_income']):
        row['monthly_income'] = monthly_income_median[row['income_type']]
    return row
# Применяем к каждой строке функцию pushNa
data = data.apply(pushNa, axis=1)


Рассмотрим отдельно данные о трудовом стаже

In [16]:
data['employment_term'].describe()

count     21525.000000
mean      66914.728907
std      131822.719320
min          24.141633
25%        1025.608174
50%        2609.841015
75%       66914.728907
max      401755.400475
Name: employment_term, dtype: float64

Анализировать стаж в днях не очень удобно, переведем его в годы.

In [17]:
data['employment_term_years'] = data['employment_term'] / 365

In [18]:
# Проверим результат
data['employment_term_years'].describe()

count    21525.000000
mean       183.328024
std        361.158135
min          0.066141
25%          2.809885
50%          7.150249
75%        183.328024
max       1100.699727
Name: employment_term_years, dtype: float64

Данные по стажу явно приведены не корректно, средняя по стажу составляет 183 года, что невозможно физически. Необходимо будет запросить дополнительную информацию, как формировалась данная статистика, чтобы понять возможную причину искажения данных.


Посмотрим на столбцы 'education_level' и 'education_level_id'


In [19]:
# Выведем уникальные значения 'education_level' и 'education_level_id'
data['education_level'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [20]:
data['education_level_id'].unique()

array([0, 1, 2, 3, 4])

Очевидно, что данные дублируют друг друга, но из-за разного регистра букв в столбце 'education_level' кажется, что уровней образования больше, надо это исправить.

In [21]:
data['education_level'] =data['education_level'].str.lower()
data['education_level'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Рассмотрим аналогичным образом столбцы 'family_status', 'family_status_id'

In [22]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [23]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

Расхождений нет,но для "красоты" можно также перевести все буквы в нижний регистр

In [24]:
data['family_status'] =data['family_status'].str.lower()
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

### Вывод

В результате предварительной обработки данных были переименованы столбцы, заполнены пропуски, зачищены артефакты, данные приведены к единому регистру. Также выявлены некоторые логические несоответсвия в данных, на которые нужно будет обратить внимание при следующих этапах анализа, или запросить дополнительную информацию (данные о стаже - средний стаж составляет 183 года и возрасте клиентов - в 101 строке возраст равен 0)

### Замена типа данных

In [25]:
data['employment_term'] = data['employment_term'].astype('int')
data['monthly_income'] = data['monthly_income'].astype('int')
data['employment_term_years'] = data['employment_term_years'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
number_of_children       21525 non-null int64
employment_term          21525 non-null int64
age                      21525 non-null int64
education_level          21525 non-null object
education_level_id       21525 non-null int64
family_status            21525 non-null object
family_status_id         21525 non-null int64
gender                   21525 non-null object
income_type              21525 non-null object
arrear_status            21525 non-null int64
monthly_income           21525 non-null int64
credit_purpose           21525 non-null object
employment_term_years    21525 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


### Вывод

Произвели замену вещественного типа данных на целочисленный с помощью метода .astype(). Данный метод используется для приведения объекта pandas к указанному dtype. astype()

### Обработка дубликатов

Проверим есть ли дубликаты, и если есть, удалим их с помощью метода drop.duplicates().При помощи этого метода строки, имеющие одно и то же имя, удаляются, и возвращается новый фрейм данных.


In [26]:
data.duplicated().sum()

71

In [27]:
data = data.drop_duplicates()

In [28]:
data.duplicated().sum()

0

### Вывод

При проверке было выявлено и удалено 71 дубликатов. Скорее всего данные были изначально были заполнены некорректно, с пропусками, которые при обработке были заменены на однотипные значения.

### Лемматизация

В таблице есть данные о целях получения кредита - столбец 'credit_purpose'. Некоторые цели однотипны, но записаны разными словами. Нужно определить количество уникальных лемм и провести категоризацию.

Выведем весь список уникальных целей

In [29]:
unique_purposes = data['credit_purpose'].value_counts().index.tolist()
data['credit_purpose'].value_counts().to_frame()


,credit_purpose
свадьба,791
на проведение свадьбы,768
сыграть свадьбу,765
операции с недвижимостью,675
покупка коммерческой недвижимости,661
операции с жильем,652
покупка жилья для сдачи,651
операции с коммерческой недвижимостью,650
жилье,646
покупка жилья,646


Определим количество уникальных лемм используя библиотеку РуMystem

In [30]:
m = Mystem()
list_of_lemmas = []
for element in data['credit_purpose']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)


[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

Обработаем получившийся список следующим образом: убираем предлоги, пробелы,'\n', а также отсортируем по убыванию и оставим 10 самых популярных значений

In [31]:
unique_purposes_list = [a for a in sorted(unique_lemmas, key=unique_lemmas.get, reverse=True)
                   if len(a) > 4 if a != ' ' if a != '\n'][0:10]
print(unique_purposes_list)


['недвижимость', 'покупка', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба', 'строительство', 'высокий', 'получение']


Уберем из списка слова, ничего не говорящие нам о цели кредита

In [32]:
unique_purposes_list.remove('покупка')
unique_purposes_list.remove('операция')
unique_purposes_list.remove('высокий')
unique_purposes_list.remove('получение')

print(unique_purposes_list)

['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба', 'строительство']


### Вывод

В результате лемматизации стало ясно, что в основном люди приходят в банк, чтобы получить кредит на приобретение жилья, приобретение автомобиля, организацию свадьбы и получение образования.

### Категоризация данных

Используем полученные в результате лемматизации цели кредитов для создания категоризации по целям получения кредита


In [33]:
realty = ['недвижим', 'жиль', 'строитель']
auto = ['авто']
wedding = ['свадь']
education = ['образов']


data['credit_purpose_category'] = 0
def cat(list_of_words, category):
  join = '|'.join(list_of_words)
  index = data[data['credit_purpose'].str.lower().str.contains(join)].index.to_list()
  for i in index:
    data.loc[i, 'credit_purpose_category'] = category
  return data

cat(wedding, 'свадьба')
cat(realty, 'недвижимость')
cat(auto, 'авто')
cat(education, 'образование')


Counter(data['credit_purpose_category'])


Counter({'недвижимость': 10811,
         'авто': 4306,
         'образование': 4013,
         'свадьба': 2324})

В таблице уже есть выделенные категории 'education_level', 'education_level_id','family_status', 'family_status_id'

In [34]:
data.groupby(['education_level', 'education_level_id']).size().to_frame('count').reset_index()

,education_level,education_level_id,count
0,высшее,0,5250
1,начальное,3,282
2,неоконченное высшее,2,744
3,среднее,1,15172
4,ученая степень,4,6


In [35]:
data.groupby(['family_status', 'family_status_id']).size().to_frame('count').reset_index()

,family_status,family_status_id,count
0,в разводе,3,1195
1,вдовец / вдова,2,959
2,гражданский брак,1,4151
3,женат / замужем,0,12339
4,не женат / не замужем,4,2810


Создадим словари для образования и семейного статуса. Для этого выведем сводные таблицы, где index - ключ словаря, а values - его значения и применим к таблице метод .to_dict().В результате у нас получится словарь со значениями

In [36]:
data.pivot_table(index = 'family_status', values = 'family_status_id').to_dict()['family_status_id']

{'в разводе': 3,
 'вдовец / вдова': 2,
 'гражданский брак': 1,
 'женат / замужем': 0,
 'не женат / не замужем': 4}

In [37]:
data.pivot_table(index = 'education_level', values = 'education_level_id').to_dict()['education_level_id']

{'высшее': 0,
 'начальное': 3,
 'неоконченное высшее': 2,
 'среднее': 1,
 'ученая степень': 4}

Нужно также выделить категории по возрасту, по количеству детей и по уровню дохода, так как по этим параметрам будет идти дальнейший анализ данных.

по возрасту ('age'):
 - 'дети'(до 18 лет)
 - 'взрослые'(18-65 лет)
 - 'пожилые'(больше 65 лет)

по количеству детей ('number_of_children'):
 - 'нет детей' (0 детей)
 - '1-2 ребенка'
 - 'многодетные'(>= 3 детей)
 
по размеру месячного дохода('monthly_income'). Чтобы определить разбивку по категориям посмотрим на квантили. Выделим следующие категории по квантилям
 - 'низкий уровень дохода' (до 107623.00)
 - 'средний уровень дохода'(от 107623.00 до 145017.00)
 - 'высокий уровень дохода''(от 145017.00 до 195813.25)
 - 'очень высокий уровень дохода (свыше 195813.25)


In [38]:
data['monthly_income'].quantile([0.25,0.5,0.75])

0.25    107623.00
0.50    142594.00
0.75    195820.25
Name: monthly_income, dtype: float64

In [39]:
def age_cat(row):
    
    if row['age'] < 18:
        return 'дети'
    elif 18 <= row['age'] < 65:
        return 'взрослые'
    else:
        return 'пожилые'
    
data['age_cat'] = data.apply(age_cat, axis=1)
    
def number_of_children_cat(row):
    if row['number_of_children'] == 0:
        return 'нет детей'
    elif row['number_of_children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'  
    
data['num_kids_count_cat'] = data.apply(number_of_children_cat, axis=1)

def monthly_income_cat(row):
    if row['monthly_income'] < 107623.00:
            return 'низкий уровень дохода'
    elif 107623.00 <= row['monthly_income'] < 145017.00:
            return 'cредний уровень дохода'
    elif 145017.00 <= row['monthly_income'] < 195813.25:
            return 'высокий уровень дохода'
    else:
            return ' очень высокий уровень дохода'  
    
data['monthly_income_cat'] = data.apply(monthly_income_cat, axis=1)

data.head(10)

,number_of_children,employment_term,age,education_level,education_level_id,family_status,family_status_id,gender,income_type,arrear_status,monthly_income,credit_purpose,employment_term_years,credit_purpose_category,age_cat,num_kids_count_cat,monthly_income_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,недвижимость,взрослые,1-2 ребенка,очень высокий уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,авто,взрослые,1-2 ребенка,cредний уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,недвижимость,взрослые,нет детей,высокий уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,образование,взрослые,многодетные,очень высокий уровень дохода
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,свадьба,взрослые,нет детей,высокий уровень дохода
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2,недвижимость,взрослые,нет детей,очень высокий уровень дохода
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7,недвижимость,взрослые,нет детей,очень высокий уровень дохода
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0,образование,взрослые,нет детей,cредний уровень дохода
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18,свадьба,взрослые,1-2 ребенка,низкий уровень дохода
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5,недвижимость,взрослые,нет детей,cредний уровень дохода


### Вывод

Мы провели категоризацию по следующим параметрам:
- 'education' и 'family_status_id' (исходные данные уже имеют категоризацию);
- 'age';
- 'children_number'
- 'monthly_income'


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Зависимость можно определить несколькими путями, можно использовать groupby(), можно выводить данные используя сводные таблицы. Попробуем оба метода

In [40]:
data_g_by_arrear = data.loc[lambda df: df['arrear_status'] == 1].groupby('num_kids_count_cat')['arrear_status'].aggregate('count')/ data.groupby('num_kids_count_cat')['arrear_status'].aggregate('count')
data_g_by_arrear

num_kids_count_cat
1-2 ребенка    0.092654
многодетные    0.081579
нет детей      0.075438
Name: arrear_status, dtype: float64

In [41]:
data_pivot_children = data.pivot_table(columns=['arrear_status'],index='num_kids_count_cat',values = 'credit_purpose', aggfunc='count')
data_pivot_children['ratio'] = data_pivot_children[1] / (data_pivot_children[1] + data_pivot_children[0]) * 100
data_pivot_children

arrear_status,0,1,ratio
num_kids_count_cat,,,
1-2 ребенка,6336,647,9.265359
многодетные,349,31,8.157895
нет детей,13028,1063,7.543822


### Вывод

Зависимость между наличием детей и возвратом кредита в срок есть. Люди, у которых нет детей, реже имеют задолженность по кредитам. При этом многодетные родители лучше соблюдают свои обязательства, чем те, у кого один-двое детей. Возможно, это результат того, что количество многодетных в данной выборке невелико по сравнению с другими категориями. Неравномерность выборки можно сразу увидеть в сводной таблице, поэтому при дальнейшем анализе данных будем использовать только сводные таблицы.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [42]:
data_pivot_family = data.pivot_table(columns=['arrear_status'],index='family_status',values = 'credit_purpose', aggfunc='count')
data_pivot_family['ratio'] = data_pivot_family[1] / (data_pivot_family[1] + data_pivot_family[0]) * 100
data_pivot_family.sort_values('ratio')

arrear_status,0,1,ratio
family_status,,,
вдовец / вдова,896,63,6.569343
в разводе,1110,85,7.112971
женат / замужем,11408,931,7.545182
гражданский брак,3763,388,9.347145
не женат / не замужем,2536,274,9.750890


### Вывод

Между семейным положением и возвратом кредита в срок также наблюдается зависимость.
Больше всего задолженностей по кредиту у тех, кто не состоит в браке вообще, или состоит в гражданском браке. Люди в браке, и разведенные возвращают кредиты лучше, чем холостые или состоящие в неофициальных отношениях. Меньше всего задолженностей по кредиту у вдов и вдовцов. Таким образом мы видим,что наиболее надежные заемщики, это те, кто находится или когда-либо был в браке. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [43]:
data_pivot_income = data.pivot_table(columns=['arrear_status'],index='monthly_income_cat',values = 'gender', aggfunc='count')
data_pivot_income['ratio'] = data_pivot_income[1] / (data_pivot_income[1] + data_pivot_income[0]) * 100
data_pivot_income.sort_values('ratio')

arrear_status,0,1,ratio
monthly_income_cat,,,
очень высокий уровень дохода,4982,383,7.138863
низкий уровень дохода,4937,427,7.960477
высокий уровень дохода,4537,421,8.491327
cредний уровень дохода,5257,510,8.843419


### Вывод

Есть небольшая разница между тем, как возвращают кредиты люди с разным уровнем дохода. Самые богатые и самые бедные возвращают деньги лучше чем люди со средним и высоким доходом. 

- Как разные цели кредита влияют на его возврат в срок?

In [44]:
data_pivot_purpose = data.pivot_table(columns=['arrear_status'],index='credit_purpose_category',values = 'gender', aggfunc='count')
data_pivot_purpose ['ratio'] = data_pivot_purpose [1] / (data_pivot_purpose [1] + data_pivot_purpose [0]) * 100
data_pivot_purpose .sort_values('ratio')

arrear_status,0,1,ratio
credit_purpose_category,,,
недвижимость,10029,782,7.233373
свадьба,2138,186,8.003442
образование,3643,370,9.220035
авто,3903,403,9.359034


### Вывод

Кредиты на покупку недвижимости берут чаще всего и возвращают лучше,чем кредиты на другие цели. Чуть хуже возвращают кредиты на проведение свадеб.Кредиты на приобретение автомобиля и получение образования возвращают хуже всего.

### Шаг 4. Общий вывод

## На возврат кредита в срок влияют разные факторы, такие как: количество детей, семейное положение заемщика, цель кредита и уровень дохода.
Согласно проведенному анализу, с наибольшей вероятностью кредит вернет заемщик с семейным статусом вдовец / вдова,не имеющий детей, взявший кредит на покупку недвижимости и имеющий очень высокий уровень дохода.
Наиболее неблагонадежный заемщик - холостой родитель 1-2 детей, со средним уровнем дохода, берущий кредит на покупку автомобиля.